In [2]:
import pandas as pd
import re
import random
import typing

In [4]:
propl_dir = "data/PropL/backtrack/"
propl_train_df = pd.read_csv(propl_dir + 'train.csv')
propl_test_df = pd.read_csv(propl_dir + 'test_id.csv')
propl_df = pd.concat([propl_train_df, propl_test_df])
propl_df.head()

,data_id,input,output
0,297653379296382230049570372147,state_0:\np1 p2 p3 p4 p5 : Prop\n⊢ True ∨\n ...,state_0_tactic_0:\napply Or.inl\nstate_1:\ncas...
1,331537354190064660954325836804,state_0:\np1 p2 p3 p4 p5 : Prop\n⊢ False → (((...,state_0_tactic_0:\nintro h1\nstate_1:\np1 p2 p...
2,331048400301130828895567840288,state_0:\np1 p2 p3 p4 p5 : Prop\n⊢ False → ((p...,state_0_tactic_0:\nintro h1\nstate_1:\np1 p2 p...
3,248900341282719602043835106617,state_0:\np1 p2 p3 p4 p5 : Prop\n⊢ (p3 ∨ p5) ∨...,state_0_tactic_0:\napply Or.inl\nstate_1:\ncas...
4,226095515506330039377011765967,state_0:\np1 p2 p3 p4 p5 : Prop\n⊢ ((True ∨ p3...,state_0_tactic_0:\napply Or.inl\nstate_1:\ncas...


In [5]:
class Proof:
  def __init__(self, statement):
    self.statement = statement[len('state_0:\n'):]
    self.state_tactic_dict = {}
    
  def add(self, tactic: str, state: str):
    tactic_list = tactic.split('\n')
    # убираем строчку типа state_1_tactic_0:
    tactic_list = tactic_list[1:]
    tactic = '\n'.join(tactic_list)
    
    state_list = state.split('\n')
    state_name = state_list[0]
    n_state = int(state[len('state_') : len(state_name) - len(':')])
    
    self.state_tactic_dict[n_state] = tactic
  
  def no_solution(self, no_solution_line: str):
    line_list = no_solution_line.split()
    prefix_list = 'no solution, return to state'.split()
    n_return_state = int(line_list[len(prefix_list)])
    for n in list(self.state_tactic_dict):
      if n > n_return_state:
        del self.state_tactic_dict[n]
  
  def get(self) -> list[str]:
    proof = list(self.state_tactic_dict.copy().values())
    return [self.statement] + proof

In [6]:
def proccess_proof(statement: str, text: str) \
  -> tuple[list[list[str]], list[str]]:
    
  # на вход берет решение методом проб и ошибок из Propl
  # на выход дает список неверных доказательств и одно верное
  
  tactic = ''
  state = ''
  text_list = text.split('\n')
  failed_proofs_list = []
  correct_proof = []
  
  start_flag = True
  tactic_flag = False
  state_flag = False
  current_proof = Proof(statement)
  
  for line in text_list:
    if 'complete' in line:
      current_proof.add(tactic, state)
      correct_proof = current_proof.get()
      
    elif 'no solution' in line:
      current_proof.add(tactic, state)
      failed_proof = current_proof.get()
      failed_proofs_list.append(failed_proof)
      current_proof.no_solution(line)
      start_flag = True
      tactic_flag = False
      state_flag = False
    
    elif 'tactic' in line:
      if not start_flag:
        #print('\n\n\n', tactic, '\n', state)
        current_proof.add(tactic, state)
      tactic = line
      start_flag = False
      tactic_flag = True
      state_flag = False
        
    elif 'state' in line:
      state = line
      state_flag = True
      tactic_flag = False
      
    elif tactic_flag:
      tactic += '\n' + line
      
    elif state_flag:
      state += '\n' + line
      
    else:
      assert False, 'неожиданная строчка'
      
  return failed_proofs_list, correct_proof

In [7]:
id_list = []
proof_list = []
correct_list = []

for _, row in propl_df.iterrows():
  failed_proofs_list, correct_proof = proccess_proof(
    row['input'], row['output']
  )
  assert len(failed_proofs_list) == row['output'].count('no solution'),\
    'неверное количество неправильных доказательств'
    
  n_failed = len(failed_proofs_list)
  n_correct = 1 if correct_proof else 0
  id_list += [row['data_id']] * (n_failed + n_correct)
  correct_list += [False] * n_failed + [True] * n_correct
  proof_list += failed_proofs_list + [correct_proof]
  
df = pd.DataFrame({
  'data_id' : id_list,
  'proof' : proof_list,
  'is_correct' : correct_list
})

In [8]:
# минус 1 тк не включаем постановку задачи
df['proof_len'] = df['proof'].apply(len) - 1

### Рассмотрим части доказательств длины N, такие, что конец доказательства / неуспешной попытки как минимум на m тактики дальше

In [9]:
N = 5
m = 2


def cut(proof: list[str]) -> list[str]:
  # + 1, считая постановку задачи 
  return proof[: N + 1]

# обрезка доказательства до части
df = df[df['proof_len'] >= N + m]
df['proof_part'] = df['proof'].apply(cut)


# убираем неправильные док-ва, совпадающие с правильными
correct_proofs = df[df['is_correct']].set_index('data_id')['proof'].to_dict()
df = df[
  df['is_correct'] |
  (~df['is_correct'] & (df['proof'] != df['data_id'].map(correct_proofs)))
]

In [10]:
df['is_correct'].value_counts()

True     16082
False    12347
Name: is_correct, dtype: int64

In [19]:
count_true = df['is_correct'].sum()
count_false = len(df) - count_true

extra = count_true - count_false
true_indices = df[df['is_correct']].index
drop_indices = pd.Series(true_indices).sample(extra, random_state=42)
df = df.drop(drop_indices)

# Чередуем строки из df_true и df_false
# чтобы было удобно делить на выборки
interleaved = []
for i in range(count_false):
    interleaved.append(df[df['is_correct']].iloc[i])
    interleaved.append(df[~df['is_correct']].iloc[i])

df = pd.DataFrame(interleaved)
df.head()

,data_id,proof,is_correct,proof_len,proof_part
108743,10137740601197937323927000320,[p1 p2 p3 p4 p5 : Prop\n⊢ ((False ∨ ((((p4 ∧ (...,True,7,[p1 p2 p3 p4 p5 : Prop\n⊢ ((False ∨ ((((p4 ∧ (...
38555,101901011237190232879400816388,[p1 p2 p3 p4 p5 : Prop\n⊢ ((((p5 → p1) → (((p1...,False,8,[p1 p2 p3 p4 p5 : Prop\n⊢ ((((p5 → p1) → (((p1...
22792,10147402779818504157948999715,[p1 p2 p3 p4 p5 : Prop\n⊢ ((False ∨ (p4 → p2 ∨...,True,9,[p1 p2 p3 p4 p5 : Prop\n⊢ ((False ∨ (p4 → p2 ∨...
179977,102028860579639362150235400604,[p1 p2 p3 p4 p5 : Prop\n⊢ ((((((((True ∧ p1) ∧...,False,11,[p1 p2 p3 p4 p5 : Prop\n⊢ ((((((((True ∧ p1) ∧...
42434,101865771099929866782122861458,[p1 p2 p3 p4 p5 : Prop\n⊢ (((p2 ∧ ((False → Fa...,True,8,[p1 p2 p3 p4 p5 : Prop\n⊢ (((p2 ∧ ((False → Fa...


In [21]:
df.to_csv("data/correct_incorrect_df.csv", index=False)